<a href="https://colab.research.google.com/github/Sicily-F/cagedbirdID/blob/main/5_Dockerfile_and_Singularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Using Dockerfile and Singularity to speed up the run time of machine learning models

## Computing resources at universities
Some scripts are easy to run locally. However, some machine learning learning models can take a long time to run. Because a key part of my thesis has been experimenting with different networks, being able to get the results from this quickly is a big bonus. Most universities (especially in the Global North) have high performance computing systems. These usually require users to download the [PuTTY](https://www.putty.org/) software. PuTTY connects users to high-performance computing (HPC) systems via a secure shell (SSH) gateway, usually using the username of your university email and the same centralised password, which allows you to log into a Linux terminal where you can directly run code in the command line or via a scheduling manager such as SLURM. Alternatively, if you are the lucky owner of a unix-based machine (MacOS, Linux), you can connect to HPC clusters using OpenSSH, which comes pre-installed on some operating systems.

## NVIDIA and Tensorflow GPU

Dockerfiles are used to create images. This image is then used as a template (or base), which can be used it to run an application (<- SW: revise wording?). The application needs an isolated environment in which to run – a container.The source code, files, dependencies, and packages are what makes up a container- in our case this will be Python and associated packages.

NVIDIA, the company which invented the [Graphical Processing Unit](https://www.itpro.co.uk/hardware/30399/what-is-a-gpu) (GPU), created many Docker images which have different versions of CUDA / cudnn. CUDA / cudnn are NVIDIA authored libraries that perform fast and parallel linear algebra computations on GPUs. Modern deep learning libraries such as Tensorflow and Pytorch rely on these libraries! Docker really comes into its own compared to Python virtual environments as it allows for the packaging of Python AND non-Python software dependencies, which makes code much easier to run on different systems.Then the way docker works is you can specify an image as a base to build on top of, and install other stuff - so in this case whatever versions of python you need and whatever packages you need. Eg. this is my build that uses the base nvidia image with cuda 11.0 and cudnn 8. And you can lob a pip install in there too. This is a work one and it runs on K80s for suuuuure. https://www.kent.ac.uk/hpc/

## Why Docker cannot be used on shared services
Since we were running code on a HPC platform, the server administraters are unable to install Docker on the HPC services. This is largely due to the security requirements of a shared service. To be able to install Docker, this would require me to have “root access” (being an admin of the server). Singularity is a Linux container platform optimized for High Performance Computing (HPC) and Enterprise Performance Computing (EPC), and can be installed without having superuser privileges, which Docker requires. Singularity is a container runtime, like Docker, but it preserves security restrictions on a container. Singularity works especially well for command line applications; see a [full](https://www.reddit.com/r/docker/comments/7y2yp2/why_is_singularity_used_as_opposed_to_docker_in/) explanation here. 

## GPU Issues
At the time our the code was written, tensorflow 2.4 was the latest version, and could only be built with CUDA 11.0. The issue was that the driver on the university HPC was NVIDIA Driver Version: 460.27.04 CUDA Version: 11.2. Further, the NVIDIA  deep learning containers (referenced below) are  not compatible with the NVIDA Tesla K80 GPU. The Tesla GPUs are now known as the Data Center GPU's, which are commonly used on university HPCs. 

## SLURM
In this context, we used the SLURM scheduling manager to submit jobs with memory/GPU requirements. [SLURM](https://slurm.schedmd.com/overview.html) is a job scheduling system for large and small Linux clusters, which allocates exclusive and/or non-exclusive access to resources (compute nodes) to users for some duration of time so they can perform work.

Most universities will have their own HPC guidance, which will be specific to your institution, so please check with your Computing department first on ways to access this.


# How to download and run a Docker image, using Singularity. 

Commands to pull an image hosted on Docker are structured in the following way: OWNER/REPOSITORY/IMAGE_NAME:TAG_NAME

In another Github [repository](https://github.com/sam-watts/tf-docker), the code for the initial DockerFile can be found. We built a Docker image which comprises of an NVIDIA base with cuda 11.0 and cudnn 8, tensorflow 2.4. and other [packages](https://github.com/sam-watts/tf-docker/blob/master/requirements.txt). If you are using a HPC, then it likely already has Singularity installed on it, if not, you can follow this [guidance](https://sylabs.io/guides/3.0/user-guide/installation.html) to install. The DockerFile tells Singularity what to build in order to run the container when you specify it. To download our custom-built base image, in your command line or once you have logged onto PuTTY, you can run:
```
singularity pull tf-docker.sif docker://swatts999/tf-docker:2.4
```
Here, swatts999 is the repository, tf-docker is the name of the image and the tag is 2.4, which corresponds to the version of Tensorflow which we used. Once complete, the .sif file will be what is specified when you want to run any Python code. The SIF file stands for singularity image format, which are [compressed and immutable making them the best choice for reproducible, production-grade containers](https://singularity-tutorial.github.io/03-building/#:~:text=Singularity%20can%20build%20containers%20in,reproducible%2C%20production%2Dgrade%20containers.).
```
singularity run --nv tf-docker.sif python 19speciesaugre.py
```
To leverage NVIDIA GPUs, you must use the Singularity flag `--nv`when running the containers. The piece of code which says 'python 19speciesaugre.py' tells the program python to run the script named 19speciesaugre.py.

You could also execute the same command in base PuTTY terminal, or on a general Linux terminal, if you have Docker installed. However, if you still want to use Docker to pull our image, then you can use this code in your command line.
```
docker pull swatts999/tf-docker:2.4
```


## Alternative options for Docker users
* NVIDIA have also built a tensorflow docker image (nvcr.io/nvidia/tensorflow:21.02-tf2-py3 also tensorflow 2.4), which is ideal for use if you are on a personal machine with up-to-date GPU's such as the NVIDIA RTX 3090. This is pre-built from source with CUDA 11.2, this image can be found [here](https://docs.nvidia.com/deeplearning/frameworks/tensorflow-release-notes/running.html#running).
* Tensorflow also have a separate Docker [image](https://www.tensorflow.org/install/docker).
* There are also some alternative Docker options for running on HPC's such as that provided by [Princeton](https://researchcomputing.princeton.edu/support/knowledge-base/tensorflow#utilization
), [Queen Mary University](https://docs.hpc.qmul.ac.uk/apps/ml/tensorflow/), [Cambridge University](https://docs.hpc.cam.ac.uk/hpc/software-packages/tensorflow.html) and the [University of Sheffield](https://docs.hpc.shef.ac.uk/en/latest/sharc/software/apps/tensorflow.html).
* If none of these work for you there are some other suggestions [here](https://github.com/tensorflow/tensorflow/issues/46093#issuecomment-787044411)
